<a href="https://colab.research.google.com/github/Angelawork/AI4Good_LiteraLingo_M3Project/blob/main/FineTuning/SHHP_Metaphor_by_Chantal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sarcasm Hyperbole Humor Paraphrase Model
https://huggingface.co/figurative-nlp/sarcasm-hyperbole-humor-paraphrase

Chantal

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!pip install transformers
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm import tqdm #gives a completion bar for your loop

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.8 MB/s eta 0:00:00


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
import transformers

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "figurative-nlp/sarcasm-hyperbole-humor-paraphrase"

from transformers import AdamW

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# Initialize the optimizer
optimizer = AdamW(model.parameters(), lr=1e-3)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config, AdamW
from sklearn.model_selection import train_test_split

# Load the dataset from a CSV file
dataset_path = '/content/Metaphor.csv'
df = pd.read_csv(dataset_path)[1500:2000]

# Split the dataset into training, validation, and test sets
train_ratio = 0.8  # Percentage of data for training set
val_ratio = 0.1    # Percentage of data for validation set
test_ratio = 0.1   # Percentage of data for test set

# Split dataset into train and remaining (val + test)
train_df, remaining_df = train_test_split(df, train_size=train_ratio, random_state=42)

# Split the remaining dataset into validation and test sets
val_df, test_df = train_test_split(remaining_df, train_size=val_ratio/(val_ratio + test_ratio), random_state=42)

# Print the sizes of the resulting datasets
print(f"Training set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")

Training set size: 400
Validation set size: 50
Test set size: 50


In [ ]:
# Set up device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
# Define a custom dataset for CSV data
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        input_text = row['Figurative sentence']
        target_text = row['Literal meaning']

        # Tokenize the input and target text
        input_tokens = self.tokenizer.encode_plus(input_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')
        target_tokens = self.tokenizer.encode_plus(target_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')

        return {
            'input_ids': input_tokens['input_ids'].squeeze(),
            'attention_mask': input_tokens['attention_mask'].squeeze(),
            'target_ids': target_tokens['input_ids'].squeeze(),
            'target_attention_mask': target_tokens['attention_mask'].squeeze()
        }

In [ ]:
# Define the dataset and data loader for training set
train_dataset = CustomDataset(train_df, tokenizer, max_length=512)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [ ]:
!pip install bert-score
import bert_score

In [ ]:
import logging
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

In [ ]:
!pip install git+https://github.com/google-research/bleurt.git
import bleurt
from bleurt import score
bleurt_scorer = score.BleurtScorer()

In [ ]:
# Set training parameters
from torch.nn import L1Loss, MSELoss
l1_strength = 1e-5
l2_strength = 1e-5

num_epochs = 4
learning_rate = 1e-4
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
bert_scores = []
bleurt_scores = []

# Fine-tuning loop for training set
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target_ids = batch['target_ids'].to(device)
        target_attention_mask = batch['target_attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=target_ids, decoder_attention_mask=target_attention_mask)
        loss = outputs.loss

        # Calculate L1 and L2 regularization
        l1_loss = 0
        l2_loss = 0
        for param in model.parameters():
            l1_loss += torch.sum(torch.abs(param))
            l2_loss += torch.sum(torch.square(param))
        regularization_loss = l1_strength * l1_loss + l2_strength * l2_loss
        loss += regularization_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        logits = outputs.logits
        predicted_ids = torch.argmax(logits, dim=-1)

        # Convert the predicted and target ids to text
        predicted_texts = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)
        target_texts = tokenizer.batch_decode(target_ids, skip_special_tokens=True)

        #print(f"predicted_texts: {predicted_texts}")

        # Calculate BERT score
        _, _, bert_score_f1 = bert_score.score(predicted_texts, target_texts, lang="en", verbose=False)
        bert_scores.append(round(bert_score_f1[0].tolist(),4))

        bleurt_scores.append(bleurt_scorer.score(references=target_texts, candidates=predicted_texts))

    accuracy = np.mean(bert_scores)
    accuracy2 = np.mean(bleurt_scores)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss}, Accuracy (BERT score): {accuracy}, Accuracy (Bleurt score): {accuracy2}')




/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


KeyboardInterrupt: ignored

In [ ]:
# Define the dataset and data loader for validation set
val_dataset = CustomDataset(val_df, tokenizer, max_length=512)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)

# Evaluation loop for validation set
model.eval()
total_val_loss = 0

with torch.no_grad():
    for batch in val_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target_ids = batch['target_ids'].to(device)
        target_attention_mask = batch['target_attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=target_ids, decoder_attention_mask=target_attention_mask)
        loss = outputs.loss

        total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_dataloader)
    print(f'Validation Loss: {avg_val_loss}')

Validation Loss: 0.017301533120469406


In [ ]:
# Define the dataset and data loader for test set
test_dataset = CustomDataset(test_df, tokenizer, max_length=512)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Evaluation loop for test set
model.eval()
total_test_loss = 0
bert_scores = []
bleurt_scores = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target_ids = batch['target_ids'].to(device)
        target_attention_mask = batch['target_attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=target_ids, decoder_attention_mask=target_attention_mask)
        loss = outputs.loss

        total_test_loss += loss.item()

        logits = outputs.logits
        predicted_ids = torch.argmax(logits, dim=-1)

        # Convert the predicted and target ids to text
        input_texts = tokenizer.batch_decode(input_ids, skip_special_tokens=True)
        predicted_texts = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)
        target_texts = tokenizer.batch_decode(target_ids, skip_special_tokens=True)

        # Calculate BERT score
        _, _, bert_score_f1 = bert_score.score(predicted_texts, target_texts, lang="en", verbose=False)
        bert_scores.append(round(bert_score_f1[0].tolist(),4))
        bleurt_scores.append(bleurt_scorer.score(references=target_texts, candidates=predicted_texts))
        print(f"input_texts: {input_texts} predicted_texts: {predicted_texts}, target_texts: {target_texts}")

    accuracy = np.mean(bert_scores)
    accuracy2 = np.mean(bleurt_scores)
    avg_test_loss = total_test_loss / len(test_dataloader)
    print(f'Test Loss: {avg_test_loss}, Accuracy (BERT score): {accuracy}, Accuracy (Bleurt score): {accuracy2}')

input_texts: ['The knife was as sharp as a plastic spoon', 'The problem was as big as Antarctica', "The boy's lemonade stand was as successful as the 1962 New York Mets", 'The detective had to instincts of A drunken moose'] predicted_texts: ["The conversation was't mean as all.", 'The conversation was very was', "The girl's supposed filled filled not", 'The garbage was very is'], target_texts: ["The knife wasn't sharp at all.", 'The problem was large.', "The boy's lemonade stand did terribly.", 'The detective was dull-witted']
input_texts: ['He completed the race at the speed of a rocket ship to the moon', 'Her legs are tree trunks', 'The music was loud like a whisper.', 'For him, she was a rocket ship.'] predicted_texts: ['He speaks the entire is attractive', 'Her trees are preserved tops', 'The music was loud loud.', 'For was was was.'], target_texts: ['He completed the race very quickly', 'Her legs are fat', 'The music was very quiet.', 'She keeps him free.']
input_texts: ['The wall

/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)


In [ ]:
np.mean(bleurt_scores)

0.2148974856780842

In [ ]:
_, _, bert_score_f1=bleurt_scorer.score(references=["its not good"],candidates=["its very well"], lang="en", verbose=False)
bert_score_f1[0].tolist()

[-0.9425795078277588]

##input_texts: ['The taste of agony laid thick upon my tongue', 'The dancer was boorish and uncoordinated.', 'The child was not religious', 'We streamed money into the education of our children']

##predicted_texts: ['The taste of agony ended thick upon my tongue', 'The dancer had as clumsy as a clownked', 'The child had as religious as a priestand', 'We put money into the education of our children'],

##target_texts: ['The taste of agony was thick upon my tongue', 'The dancer was as graceful as a honking goose.', 'The child was as religious as a sinner', 'We put money into the education of our children']

# Test Loss: 0.0382503722436153, Accuracy (BERT score): 0.940323076923077

#Over 1500 rows of training

# input_texts: ['The knife was as sharp as a plastic spoon', 'The problem was as big as Antarctica', "The boy's lemonade stand was as successful as the 1962 New York Mets", 'The detective had to instincts of A drunken moose']

# predicted_texts: ["The girl was't't all", 'The girl was very', "The girl's not's as not", 'The garbage was notit'],

# target_texts: ["The knife wasn't sharp at all.", 'The problem was large.', "The boy's lemonade stand did terribly.", 'The detective was dull-witted']

## Test Loss: 0.06245673591127762, Accuracy (BERT score): 0.8792692307692309, bleurt score: -0.942

Test on test dataset

In [ ]:
import os
import shutil
import torch
!pip install transformers
from transformers import AdamW, AutoModelForSeq2SeqLM, AutoTokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
name1="angel1987/SHHPmodel_metaphor"

tokenizer = AutoTokenizer.from_pretrained(name1)
model = AutoModelForSeq2SeqLM.from_pretrained(name1)

In [ ]:
# Set up device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

NameError: ignored

In [ ]:
print(train_df)

NameError: ignored

In [ ]:
model.eval()
input_ids = tokenizer("The wind floats through the leaves", padding=True, truncation=False,return_tensors="pt").input_ids.to(device)
outputs = model.generate(input_ids,num_beams = 5)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("OUT: ", result)

OUT:  The wind blows through the leaves


In [ ]:
model.save_pretrained('SHHPmodel_metaphor')
tokenizer.save_pretrained('SHHPtokenizer_metaphor')

NameError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil

source_path = "/content/SHHPmodel_metaphor/pytorch_model.bin"  # Replace with the actual path to your file
target_path = "/content/drive/MyDrive/pytorch_model.bin"  # Replace with the desired path in your Google Drive

shutil.copyfile(source_path, target_path)

'/content/drive/MyDrive/pytorch_model.bin'

In [ ]:
import os
import shutil
import torch
!pip install transformers
from transformers import AdamW, AutoModelForSeq2SeqLM, AutoTokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.5 MB/s eta 0:00:00


In [ ]:
name1="angel1987/SHHPmodel_metaphor"

tokenizer = AutoTokenizer.from_pretrained(name1)
model = AutoModelForSeq2SeqLM.from_pretrained(name1)

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import sys
!pip install bert-score
import bert_score
import numpy as np
from transformers import logging

!pip install git+https://github.com/google-research/bleurt.git
import bleurt
from bleurt import score
logging.set_verbosity_error()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-qvxbljdh
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-qvxbljdh
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.8 MB/s eta 0:00:00
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456764 sha256=46b3491fabbb45c5b18df84b0ad5ae049a9b02921fccbbedef8cbda807091ba0
  Stored in directory: /tmp/pip-ephem-wheel-cache-07ky5mm6/wheels/64/f4/2c/509a6c31b8ebde891a81029fd94f199b1b92f0e7cfc20d417a

In [ ]:
bertscore = bert_score.BERTScorer(lang='en', rescale_with_baseline=True)
#checkpoint = '/content/bleurt_config.json'
bleurt_scorer = score.BleurtScorer()
#bleurt_scorer = load("bleurt", module_type="metric")

In [ ]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Preprocessing
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Tokenize the text
    tokens = nltk.word_tokenize(text.lower())
    # Remove stop words and punctuation
    tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    # Return the preprocessed text as a string
    return ' '.join(tokens)

# Calculate cosine similarity for list of text objects
def calculate_cosine_similarity(text1, text2):
    # Preprocess the texts
    preprocessed_text1 = preprocess_text(text1)
    preprocessed_text2 = preprocess_text(text2)

    # Vectorize the texts
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([preprocessed_text1, preprocessed_text2])

    # Calculate cosine similarity
    similarity_matrix = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])
    similarity_score = similarity_matrix[0][0]

    return similarity_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import pandas as pd
def evaluate(link, amount):
  model.eval()
  idioms_df = pd.read_csv('/content/Metaphor.csv',encoding = "ISO-8859-1")
  fig = list(idioms_df["Figurative sentence"][:amount])
  lit = list(idioms_df["Literal meaning"][:amount])
  hyps=[]
  refs=lit
  counter=0
  for sentence in fig:
    counter+=1

    #insert model----------------------
    #model 1
    #input_ids = tokenizer("paraphrase the sentence : "+sentence, padding=True, truncation=False,return_tensors="pt").input_ids
    input_ids = tokenizer(sentence, padding=True, truncation=False,return_tensors="pt").input_ids
    outputs = model.generate(input_ids,num_beams = 5)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    #insert model----------------------

    if (counter%5 == 0):
      print("Generating at "+str(counter)+" times")
      print(sentence, result)
    hyps.append(result)
  return hyps, refs



def score(hyps, refs, type):
  if type=='bert':
    scores = []
    for l1,l2 in zip(hyps, refs):
      _, _, bert_score = bertscore.score([l1], [l2])
      scores.append(round(bert_score[0].tolist(),4))

  elif type=='bleu':
    scores = bleurt_scorer.score(references=refs, candidates=hyps)
    #scores = bleurt.compute(predictions=hyps, references=refs)
  elif type=='cosine':
    scores = []
    for sentence1, sentence2 in zip(hyps, refs):
          scores.append(calculate_cosine_similarity(sentence1, sentence2))
  print('The average score is {}'.format(np.mean(scores)))

In [ ]:
hyps, refs = evaluate("/content/Metaphor.csv",500)

Generating at 5 times
I want to frolic in sunshine once again I want to be in sunshine once again
Generating at 10 times
It dawned to him that she had betrayed him It was stricken to him that had betrayed him
Generating at 15 times
In the depth of the night I sprawl in the bed In the depth of the night I lie in the bed
Generating at 20 times
Life in the camp wilted him Life in the camp tired him
Generating at 25 times
His playing ensconced him as a Romantic His playing established him as a Romantic
Generating at 30 times
The smell of smoke rested on the delinquent The smell of smoke settled on the delinquent
Generating at 35 times
Their faces were shrouded with sadness Their faces were covered with sadness
Generating at 40 times
My head lay in the crook of her neck My head tops in the crook of her neck
Generating at 45 times
My alarm roared at me this morning My alarm buzzed at me
Generating at 50 times
I won't bow to reading other people's mail I won't go to reading other people's mai

In [ ]:
score(hyps, refs,'cosine')

The average score is 0.5421211912790805
